In [22]:
!pip install --quiet numpy pandas matplotlib yfinance


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go


In [23]:
# Fetch historical data
def get_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return data


In [24]:
# Calculate returns
def calculate_returns(data):
    returns = data.pct_change().dropna()
    return returns

In [25]:
# Simulate portfolios with constraints
def simulate_portfolios(returns, num_portfolios=10000, constraints=None):
    num_assets = len(returns.columns)
    results = np.zeros((3, num_portfolios))  # No need to store weights here
    portfolio_weights = []  # List to store weights of portfolios
    for i in range(num_portfolios):
        weights = np.random.random(num_assets)
        if constraints:
            weights = weights / np.sum(weights)  # Ensure weights sum to 1
            weights = np.clip(weights, 0, constraints)  # Apply constraints
            weights /= np.sum(weights)  # Re-normalize after constraints

        portfolio_return = np.sum(weights * returns.mean()) * 252  # Annualize return
        portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))  # Annualize risk
        portfolio_sharpe = portfolio_return / portfolio_stddev  # Sharpe Ratio

        results[0,i] = portfolio_return
        results[1,i] = portfolio_stddev
        results[2,i] = portfolio_sharpe
        portfolio_weights.append(weights)  # Store weights

    return results, portfolio_weights

In [26]:
# Optimize portfolios
def get_optimal_portfolios(results, portfolio_weights):
    max_sharpe_idx = np.argmax(results[2])
    min_volatility_idx = np.argmin(results[1])

    max_sharpe = results[:, max_sharpe_idx]
    min_volatility = results[:, min_volatility_idx]

    max_sharpe_weights = portfolio_weights[max_sharpe_idx]
    min_volatility_weights = portfolio_weights[min_volatility_idx]

    return max_sharpe, max_sharpe_weights, min_volatility, min_volatility_weights

In [17]:
# Plot efficient frontier with optimal portfolios
def plot_efficient_frontier(results, max_sharpe, max_sharpe_weights, min_volatility, min_volatility_weights):
    fig = go.Figure()

    # Scatter plot for all portfolios
    fig.add_trace(go.Scatter(
        x=results[1], y=results[0], mode='markers',
        marker=dict(color=results[2], colorscale='Viridis', colorbar=dict(title='Sharpe Ratio')),
        name='Portfolios'
    ))

    # Optimal portfolios
    fig.add_trace(go.Scatter(
        x=[max_sharpe[1]], y=[max_sharpe[0]], mode='markers',
        marker=dict(color='red', size=10, symbol='cross'),
        name='Max Sharpe Ratio'
    ))

    fig.add_trace(go.Scatter(
        x=[min_volatility[1]], y=[min_volatility[0]], mode='markers',
        marker=dict(color='blue', size=10, symbol='cross'),
        name='Min Volatility'
    ))

    fig.update_layout(
        title='Efficient Frontier with Optimal Portfolios',
        xaxis_title='Risk (Standard Deviation)',
        yaxis_title='Return',
        showlegend=True
    )

    fig.show()


In [21]:
# Define parameters
# tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'AMD', 'BABA']  # Example tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META']  # Example tickers
start_date = '2020-01-01'
end_date = '2023-01-01'
constraints = 0.5  # Maximum weight for any asset

# Fetch data
data = get_data(tickers, start_date, end_date)

# Calculate returns
returns = calculate_returns(data)

# Simulate portfolios
results, portfolio_weights = simulate_portfolios(returns, constraints=constraints)

# Optimize portfolios
max_sharpe, max_sharpe_weights, min_volatility, min_volatility_weights = get_optimal_portfolios(results, portfolio_weights)

# Plot efficient frontier
plot_efficient_frontier(results, max_sharpe, max_sharpe_weights, min_volatility, min_volatility_weights)

[*********************100%%**********************]  6 of 6 completed
